In [1]:
import sys,os,os.path
import tensorflow as tf
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,floatX=float32'
from keras import backend as K
sess  = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [2]:
cd 'deep-learning-models/'

/data/fs4/home/dgrossman/deep-learning-models


In [3]:
import json
import time
allFiles = 'allImages'
root = '/local_data/dgrossman/dgCars/'
af = open(allFiles,'r')
data = list()
for d in af:
    d = d.strip()
    data.append(json.loads(d))

In [4]:
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions
import numpy as np

In [5]:
s = time.time()
model = ResNet50(weights='imagenet')
print ('loadResNet50',time.time() - s)



K.image_dim_ordering: tf
102744064/102853048 [============================>.] - ETA: 0sloadResNet50 103.93724489212036


In [ ]:
good = list()
bad = list()
file = list()

In [ ]:
interesting = set()

for x in ['police_van','moving_van','car','pickup','suv','truck','crossover','van','minivan','sports_car','cab','racer','convertible','car_wheel','jeep','ambulance']:
    interesting.add(x)

count = 0
im = 0
#with tf.device('/gpu:0'):
#    print ('time:',time.time())
s = time.time()
print ('time:',time.time() - s)
if True:
    for d in data:   
        img_path = '{0}/{1}'.format(root,d['filename'])
        flag = True
        try:
            img = image.load_img(img_path, target_size=(224, 224))

        except:
            #print('FILE :',d['filename'])
            file.append(d)
            flag = False

        if flag:
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            preds = model.predict(x)
            predictions = decode_predictions(preds)[0][:4]
            #out = ''
            found = False
            for prediction in predictions:
                i,t,score = prediction
                #out = d['filename'], prediction
                if t in interesting:
                    #out = 'GOOD'+' ' + d['filename']+' '+t
                    good.append((d,t))
                    found = True
                    break
            if not found:
                bad.append((d,predictions[0][1]))
                #out = 'BAD'+ ' ' + d['filename']+ ' ' + predictions[0][1]
            #print (out)
            #print ('Predicted',decode_predictions(preds)[0][:4],' sec:',time.time() - s )

        atOnce = 10000
        if count == atOnce:
            count = 0
            im = im + 1
            z = time.time() - s
            print('processed:',im * atOnce,'Images','good',len(good),'bad',len(bad),'file',len(file),z)
            s = time.time()
        count = count + 1

time: 2.956390380859375e-05
24576/35363 [===================>..........] - ETA: 0sprocessed: 10000 Images good 6681 bad 2 file 3318 199.4498679637909
processed: 20000 Images good 14144 bad 7 file 5850 221.3894076347351
processed: 30000 Images good 22128 bad 10 file 7863 233.6485574245453


In [ ]:
print('processed:',len(good) + len(bad) + len(file),'Images','good',len(good),'bad',len(bad),'file',len(file))

In [19]:
from collections import defaultdict
q = defaultdict(int)

In [20]:
for b in good:
    q[b[1]] = q[b[1]]+1

In [21]:
for qq in q:
    print (qq,q[qq])


pickup 197430
minivan 197889
jeep 58346
car_wheel 43754
sports_car 113967
ambulance 12167
convertible 112168
racer 30318
cab 75600


In [8]:
gfile = open('foundCars','w')
    for g in good:
        gfile.write(g+'\n')
gfile.close()

IndentationError: unexpected indent (<ipython-input-8-d51da612250b>, line 2)

In [22]:
gfile = open('foundCars','w')
for g in good:
    dat, classification = g
    dat['resnet50'] = classification
    gfile.write(json.dumps(dat)+'\n')
gfile.close()


In [14]:
good[0]

({'color': 'yellow',
  'filename': 'yellow/McLaren/2d2b71d630a0267eb664b04af33debc8.jpg',
  'hash': '2d2b71d630a0267eb664b04af33debc8',
  'make': 'McLaren',
  'model': 'MP4-12C',
  'url': 'http://i.telegraph.co.uk/multimedia/archive/02370/Spider-3_2370200b.jpg'},
 'sports_car')

In [1]:
7+2

9